In [ ]:
import numpy as np
import pandas as pd
import guild.ipy as guild
import matplotlib.pyplot as plt

METRICS = ['MAE', "R2"]   

In [ ]:
runs = guild.runs()
comp = runs.compare()

In [ ]:
comp[comp.operation=="optimal"][METRICS].agg(["min", "median", "mean", "max"])

In [ ]:
comp[comp.operation=="next"][METRICS].agg(["min", "median", "mean", "max"])

In [ ]:
comp[comp.operation=="nodata"][METRICS].agg(["min", "median", "mean", "max"])

In [ ]:
comp[comp.operation=="local"][METRICS].agg(["min", "median", "mean", "max"])

In [ ]:
comp[comp.operation=="map"][METRICS].agg(["min", "median", "mean", "max"])

In [ ]:
g = comp[-comp["operation"].str.contains("\+")].groupby("operation")
r = pd.DataFrame(g["MAE"].min())
r["R2"] = g["R2"].max()
r = r.sort_values("MAE")
r["MAE M"] = r["MAE"]/1e6
r["Scenario"] = "\\"+r.index.str[0:3]
r["Future data rates"] = g["input_succDR"].max() > 0
r["Past data rates"] = g["input_prevDR"].max() > 0
r["Positions"] = g["inputs"].max().str.contains("lat, long")
r["Signal strengths"] = g["inputs"].max().str.contains("rsrq, rsrp, rssi, sinr, signal")

In [ ]:
f = r[["Scenario", "Future data rates", "Past data rates", "Positions", "Signal strengths", "MAE M", "R2"]].to_latex(
    index=False, float_format="%.2f", escape=False, column_format="lccccSS")

# Fix export problem when index is False
t = f.splitlines()
t[3], t[4] = t[4], t[3]
f = "\n".join(t)
    
# Make changes to formatting for paper
f = f.replace("True", "\checkmark").replace("False", "").replace(
        "Scenario &  Future data rates &  Past data rates &  Positions &  Signal strengths &  MAE M &   R2 \\",
        """Scenario & \multicolumn{4}{c}{Inputs} & {MAE} & {\!\!\!$R^2$\!\!\!}\\\\
           & \multicolumn{2}{c}{Data rates} & \!Positions\! & Signal & \!\!{[\\si{\\mega\\bit\\per\\second}]}\!\! & \\\\
           & \!Future\! & \!Past\! & & \!\!strengths\!\!& \\"""
    )

print(f)